# Verilerin Önizlemesi
- Her biri 4 özelliğe (çanak yaprağı uzunluğu, çanak yaprağı genişliği, taç yaprağı uzunluğu, taç yaprağı genişliği) sahip 150 gözlem vardır.
- Boş değer yoktur, dolayısıyla bu konuda endişelenmemize gerek yok.
- Her türe (setosa, versicolor, virginica) ait 50'şer gözlem bulunmaktadır.


In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/iris/Iris.csv')
df = df.drop(['Id'],axis=1)
target = df['Species']
s = set()
for val in target:
    s.add(val)
s = list(s)
rows = list(range(100,150))
df = df.drop(df.index[rows])

Iris veri setinde üç sınıf bulunduğundan sınıflardan birini kaldıracağız. Bu bizi ikili sınıf sınıflandırma problemiyle karşı karşıya bırakıyor.

In [ ]:
import matplotlib.pyplot as plt

x = df['SepalLengthCm']
y = df['PetalLengthCm']

setosa_x = x[:50]
setosa_y = y[:50]

versicolor_x = x[50:]
versicolor_y = y[50:]

plt.figure(figsize=(8,6))
plt.scatter(setosa_x,setosa_y,marker='+',color='green')
plt.scatter(versicolor_x,versicolor_y,marker='_',color='red')
plt.show()


Ayrıca kullanabileceğimiz dört özellik var. Yalnızca iki özelliği kullanacağız, yani Sepal uzunluğu ve Petal uzunluğu. Bu iki özelliği alıyoruz ve görselleştirmek için bunları çiziyoruz. Yukarıdaki grafikten veri noktalarını ayırmak için doğrusal bir çizginin kullanılabileceği sonucunu çıkarabilirsiniz.

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
## Drop rest of the features and extract the target values
df = df.drop(['SepalWidthCm','PetalWidthCm'],axis=1)
Y = []
target = df['Species']
for val in target:
    if(val == 'Iris-setosa'):
        Y.append(-1)
    else:
        Y.append(1)
df = df.drop(['Species'],axis=1)
X = df.values.tolist()
## Shuffle and split the data into training and test set
X, Y = shuffle(X,Y)
x_train = []
y_train = []
x_test = []
y_test = []

x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.9)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

y_train = y_train.reshape(90,1)
y_test = y_test.reshape(10,1)

Gerekli özellikleri çıkarıp eğitim ve test verilerine bölüyoruz. Verilerin %90'ı eğitim için, geri kalan %10'u ise test için kullanılır. Şimdi Numpy kütüphanesini kullanarak SVM modelimizi oluşturalım.

In [ ]:
## Support Vector Machine 
import numpy as np

train_f1 = x_train[:,0]
train_f2 = x_train[:,1]

train_f1 = train_f1.reshape(90,1)
train_f2 = train_f2.reshape(90,1)

w1 = np.zeros((90,1))
w2 = np.zeros((90,1))

epochs = 1
alpha = 0.0001

while(epochs < 10000):
    y = w1 * train_f1 + w2 * train_f2
    prod = y * y_train
    print(epochs)
    count = 0
    for val in prod:
        if(val >= 1):
            cost = 0
            w1 = w1 - alpha * (2 * 1/epochs * w1)
            w2 = w2 - alpha * (2 * 1/epochs * w2)
            
        else:
            cost = 1 - val 
            w1 = w1 + alpha * (train_f1[count] * y_train[count] - 2 * 1/epochs * w1)
            w2 = w2 + alpha * (train_f2[count] * y_train[count] - 2 * 1/epochs * w2)
        count += 1
    epochs += 1

α(0,0001) öğrenme hızıdır ve düzenleme parametresi λ, 1/dönem olarak ayarlanır. Bu nedenle, düzenlileştirme değeri arttıkça çağ sayısı azalır.

In [ ]:
from sklearn.metrics import accuracy_score

## Clip the weights 
index = list(range(10,90))
w1 = np.delete(w1,index)
w2 = np.delete(w2,index)

w1 = w1.reshape(10,1)
w2 = w2.reshape(10,1)
## Extract the test data features 
test_f1 = x_test[:,0]
test_f2 = x_test[:,1]

test_f1 = test_f1.reshape(10,1)
test_f2 = test_f2.reshape(10,1)
## Predict
y_pred = w1 * test_f1 + w2 * test_f2
predictions = []
for val in y_pred:
    if(val > 1):
        predictions.append(1)
    else:
        predictions.append(-1)

print(accuracy_score(y_test,predictions))

Test verileri yalnızca 10 veri noktası içerdiğinden artık ağırlıkları kırpıyoruz. Özellikleri test verilerinden çıkarıyoruz ve değerleri tahmin ediyoruz. 
Tahminleri elde edip gerçek değerlerle karşılaştırıp modelimizin doğruluğunu yazdırıyoruz.

SVM algoritmasını uygulamanın başka bir basit yolu daha var. SVM modelini uygulamak için Scikit öğrenme kütüphanesini kullanabilir ve sadece ilgili fonksiyonları çağırabiliriz. Kod satırı sayısı önemli ölçüde çok az satır azaltır.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred))